# Document Splitting

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [3]:
chunk_size = 26
chunk_overlap = 4

In [4]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

Why doesn't this split the string below?

In [5]:
text1 = 'abcdefghijklmnopqrstuvwxyz'

In [9]:
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

In [10]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'

In [11]:
r_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

Ok, this splits the string but we have an overlap specified as 5, but it looks like 3? (try an even number)

In [12]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [13]:
# ()l()m -> 4 overlap

r_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [16]:
# default sepeartor: new-line

c_splitter.split_text(text3)

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [17]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Recursive splitting details
RecursiveCharacterTextSplitter is recommended for generic text.

In [18]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [19]:
len(some_text)

496

In [20]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [21]:
c_splitter.split_text(some_text)

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [22]:
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

Let's reduce the chunk size a bit and add a period to our separators:

In [25]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related",
 '. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns',
 '. Carriage returns are the "backslash n" you see embedded in this string',
 '. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [24]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
                                #regex - look behind
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related.",
 'For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns.',
 'Carriage returns are the "backslash n" you see embedded in this string.',
 'Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [26]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

In [27]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [28]:
docs = text_splitter.split_documents(pages)

In [30]:
len(pages)

22

In [29]:
len(docs)

77

In [31]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")
notion_db = loader.load()

In [35]:
docs = text_splitter.split_documents(notion_db)

In [36]:
len(notion_db)

52

In [38]:
len(docs)

358

## Custom

In [39]:
loader = PyPDFLoader("교사인터뷰.pdf")

In [40]:
pages = loader.load()

In [43]:
pages[0].page_content

'- 1 -교대를 다니면서 학교 생활이 매일 기쁜 것만은 아니었지만 동기들도 착하고 또 교생 실습에\n서 오는 기쁨 또 이제 미래에 대한 안정감 이런 것들이 우리 딸도 교사가 되면 좋겠구나라는 \n생각을 좀 했었어요 .\n왜 아이가 원치 않아도 부모님이 교대 가라는 말 많이 하시잖아요 . \n저도 그러고 싶은 생각이 들었고 제 동기들도 그래서 교대에 왔다는 친구들도 꽤 있었어요 . \n그런데 막상 교사가 되니까 이게 딸이 원한다면 적극 추천하겠지만 원치 않는다면 안정적이라\n는 이유 등으로 억지로 강요할 수는 없을 것 같은 생각이 들어요 .\n스스로 원해서 된 것이 아니라면 제가 지금부터 말씀드리는 이 몇 가지 단점들을 극복하기 힘\n들 수도 있어요 . \n교대를 준비하시는 분들 교사라는 직업에 관심이 있는 분들 발견한 것을 몇 가지 알려드릴게\n요. \n제가 초등학교 교사 되면서 제일 힘들었던 것 중에 하나는 화장실을 가기가 너무 힘들다는 거\n예요. \n수업 시간은 당연한 거고 쉬는 시간까지 제가 화장실을 갈 수 있는 사람이라는 걸 너무 까먹\n어요.\n그냥 화장실 갈 시간도 없나 이제 쉬는 시간에 가면 되지 이런 생각하실 텐데 수업 다 끝나면 \n쉬는 시간 되면 학생들이 우르르 몰려옵니다 . \n집에 있었던 일이나 또 친구에 대해서 친구가 자기를 잘못했다 이르는 거 또 친구가 때렸다 \n이렇게 알리는 거\n이런 것들을 선생님한테 말하러 진짜 오르로 거의 줄 서 있어요 . \n줄 서지 않으라 그러면 정말 완전히 그냥 완전 시장판처럼 그렇게 올려와요 . \n그러고 나서 그러면 이제 그거를 이렇게 하나하나 응대하다 보면 10분이 굉장히 짧아요 . \n10분이 전부 쉬는 시간도 아닌 게 한 5분에서 7분이 지나면 다음 시간을 준비시켜야 되잖아\n요. \n그래서 이제 모든 에너지가 아이들에게 들어가서 제가\n화장실 갈 시간조차 못 내는 게 첫 번째 단점이에요 . \n가뭄에 콩 나듯 전담 시간이 있어요 . \n4학년 담임인데 지금 한 3시간 정도 있거든요 . \n이

In [44]:
pages[0].metadata

{'source': '교사인터뷰.pdf', 'page': 0}

In [45]:
docs = text_splitter.split_documents(pages)

In [46]:
len(pages)

22

In [50]:
len(docs)

43

In [52]:
docs[0]

Document(page_content='- 1 -교대를 다니면서 학교 생활이 매일 기쁜 것만은 아니었지만 동기들도 착하고 또 교생 실습에\n서 오는 기쁨 또 이제 미래에 대한 안정감 이런 것들이 우리 딸도 교사가 되면 좋겠구나라는 \n생각을 좀 했었어요 .\n왜 아이가 원치 않아도 부모님이 교대 가라는 말 많이 하시잖아요 . \n저도 그러고 싶은 생각이 들었고 제 동기들도 그래서 교대에 왔다는 친구들도 꽤 있었어요 . \n그런데 막상 교사가 되니까 이게 딸이 원한다면 적극 추천하겠지만 원치 않는다면 안정적이라\n는 이유 등으로 억지로 강요할 수는 없을 것 같은 생각이 들어요 .\n스스로 원해서 된 것이 아니라면 제가 지금부터 말씀드리는 이 몇 가지 단점들을 극복하기 힘\n들 수도 있어요 . \n교대를 준비하시는 분들 교사라는 직업에 관심이 있는 분들 발견한 것을 몇 가지 알려드릴게\n요. \n제가 초등학교 교사 되면서 제일 힘들었던 것 중에 하나는 화장실을 가기가 너무 힘들다는 거\n예요. \n수업 시간은 당연한 거고 쉬는 시간까지 제가 화장실을 갈 수 있는 사람이라는 걸 너무 까먹\n어요.\n그냥 화장실 갈 시간도 없나 이제 쉬는 시간에 가면 되지 이런 생각하실 텐데 수업 다 끝나면 \n쉬는 시간 되면 학생들이 우르르 몰려옵니다 . \n집에 있었던 일이나 또 친구에 대해서 친구가 자기를 잘못했다 이르는 거 또 친구가 때렸다 \n이렇게 알리는 거\n이런 것들을 선생님한테 말하러 진짜 오르로 거의 줄 서 있어요 . \n줄 서지 않으라 그러면 정말 완전히 그냥 완전 시장판처럼 그렇게 올려와요 . \n그러고 나서 그러면 이제 그거를 이렇게 하나하나 응대하다 보면 10분이 굉장히 짧아요 . \n10분이 전부 쉬는 시간도 아닌 게 한 5분에서 7분이 지나면 다음 시간을 준비시켜야 되잖아\n요. \n그래서 이제 모든 에너지가 아이들에게 들어가서 제가\n화장실 갈 시간조차 못 내는 게 첫 번째 단점이에요 . \n가뭄에 콩 나듯 전담 시간이 있어요 . \n4학년 담임인데 

In [54]:
docs[1]

Document(page_content='요. \n그래서 이제 모든 에너지가 아이들에게 들어가서 제가\n화장실 갈 시간조차 못 내는 게 첫 번째 단점이에요 . \n가뭄에 콩 나듯 전담 시간이 있어요 . \n4학년 담임인데 지금 한 3시간 정도 있거든요 . \n이것도 1~2학년 담임일 경우에는 없을 수도 있어요 . \n이게 좀 저는 단점이 아닌가 싶었어요 . \n두 번째 성취 욕구를 느끼기가 힘들어요 . \n학습지도 생활 지도 인성지도 이러는 것들이\n눈에 드러나게 뭔가 나에게 성취를 주거나 이러진 않아요 . \n이러긴 힘들어요 . 학습도 굉장히 이제 천천히 변화하는 거잖아요 . \n그럼 제가 설명했을 때 아 이러는 그런 깨달음의 소리 정도 또 생활지도는 진짜 반복을 계속\n해도 수십 번 이야기해도 이 습관을 좋은 방향으로 바꾸는 게 힘들잖아요 .\n인성 부분은 더 잘 안대요 . 학부모님도 하지 못한 인성 지도를 이렇게 교사가 눈에 띄게 하기\n는 힘들죠 . \n성취 욕구를 느낄 만큼 하는 것이 쉽지 않아요 . \n학습 생활 인성지도 이 각각의 변화라는 것은 정말 정말 미묘하고 변화된다 해도 천천히 티가', metadata={'source': '교사인터뷰.pdf', 'page': 0})

## Token splitting
We can also split on token count explicity, if we want.

This can be useful because LLMs often have context windows designated in tokens.

Tokens are often ~4 characters.

In [55]:
from langchain.text_splitter import TokenTextSplitter

In [56]:
text_splitter = TokenTextSplitter(
    chunk_size=1,
    chunk_overlap=0
)

In [57]:
text1 = "foo bar bazzyfoo"

In [58]:
text_splitter.split_text(text1)

['foo', ' bar', ' b', 'az', 'zy', 'foo']

In [59]:
text_splitter = TokenTextSplitter(
    chunk_size=10,
    chunk_overlap=0
)

## Built-in PDF

In [66]:
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

In [67]:
docs = text_splitter.split_documents(pages)

In [68]:
docs[0]

Document(page_content='MachineLearning-Lecture01  \n', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0})

In [69]:
pages[0].metadata

{'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}

In [70]:
len(docs)

1557

In [73]:
docs[:10]

[Document(page_content='MachineLearning-Lecture01  \n', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}),
 Document(page_content='Instructor (Andrew Ng):  Okay. Good', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}),
 Document(page_content=' morning. Welcome to CS229, the machine ', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}),
 Document(page_content='\nlearning class. So what I wanna do today', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}),
 Document(page_content=' is ju st spend a little time going over the', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}),
 Document(page_content=' logistics \nof the class, and then we', metadata={'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}),
 Document(page_content="'ll start to  talk a bit about machine learning", metadata={'source': 

## Custom PDF

In [76]:
loader = PyPDFLoader('교사인터뷰.pdf')
pages = loader.load()

In [78]:
docs = text_splitter.split_documents(pages)

In [79]:
docs[0]

Document(page_content='- 1 -교대�', metadata={'source': '교사인터뷰.pdf', 'page': 0})

In [80]:
docs[0].metadata

{'source': '교사인터뷰.pdf', 'page': 0}

In [81]:
docs[:10]

[Document(page_content='- 1 -교대�', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='�� 다니면', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='서 학교 �', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='��활이 매', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='일 기쁜 ', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='것만은 �', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='�니었지�', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='�� 동기�', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='�도 착하�', metadata={'source': '교사인터뷰.pdf', 'page': 0}),
 Document(page_content='�� 또 교�', metadata={'source': '교사인터뷰.pdf', 'page': 0})]

## Context aware splitting
Chunking aims to keep text with common context together.

A text splitting often uses sentences or other delimiters to keep related text together but many documents (such as Markdown) have structure (headers) that can be explicitly used in splitting.

We can use MarkdownHeaderTextSplitter to preserve header metadata in our chunks, as show below.

In [82]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [83]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n 
## Chapter 2\n\n \
Hi this is Molly"""

In [84]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [85]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [86]:
md_header_splits[0]

Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1'})

In [87]:
md_header_splits[1]

Document(page_content='Hi this is Lance', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1', 'Header 3': 'Section'})

Try on a real Markdown file, like a Notion database.

In [92]:
loader = NotionDirectoryLoader("docs/Notion_DB")
docs = loader.load()
txt = ' '.join([d.page_content for d in docs])

In [93]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [94]:
md_header_splits = markdown_splitter.split_text(txt)

In [95]:
md_header_splits[0]`

Document(page_content="Saying goodbye to Blendle (from a colleague) and to a colleague (from Blendle) is a very normal and natural thing. When done right, it can even be a beautiful thing.  \nWe advise you to read the backdrop below first, but feel free to jump in right away with the 'Here's what you can do'-section :). General note: you do not have to do this alone, so please ask for advice and help!  \n- **Backdrop**  \nSaying goodbye to Blendle (from a colleague) and to a colleague (from Blendle) is a very normal and natural thing. When done right, it can even be a beautiful thing.  \nSaying goodbye to people is also an important part of keeping your team on the right track. Firing can even be a part of your [Personnel Planning](https://www.notion.so/Hiring-451bbcfe8d9b49438c0633326bb7af0a?pvs=21). The most common situation will be when you think someone is no longer a good match with Blendle for whatever reason. This doesn't happen overnight, so try to spot situations where this is